In [ ]:
#bsic studnet


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels,predictions))

              precision    recall  f1-score   support

         1.0       1.00      0.00      0.01      1351
         2.0       0.49      0.99      0.66        98
         3.0       0.42      0.94      0.58      1121
         4.0       0.00      0.00      0.00       128

    accuracy                           0.43      2698
   macro avg       0.48      0.48      0.31      2698
weighted avg       0.69      0.43      0.27      2698



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#pruned

In [ ]:
student_model.save("/content/drive/MyDrive/DL/KD/base_cnn_student.h5")
pruned_keras_file = "/content/drive/MyDrive/DL/KD/base_cnn_student_strip.h5"
pruned_tflite_file = "/content/drive/MyDrive/DL/KD/base_cnn_student_strip_tflite.h5"
import tempfile


import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

end_step = 300*epochs
# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(student_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

predictions = np.array([])
labels =  np.array([])

for x, y in test_ds:
  predictions = np.concatenate([predictions, model_for_pruning.predict(x).argmax(axis =1)+1])
  labels = np.concatenate([labels, y.numpy()])


from sklearn.metrics import classification_report
print(classification_report(labels,predictions))


model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# _, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)


converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

# _, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


Epoch 1/2


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


304/304 [==============================] - 406s 1s/step - loss: 0.2424 - accuracy: 0.9289 - val_loss: 3.6064 - val_accuracy: 0.2006
Epoch 2/2
304/304 [==============================] - 382s 1s/step - loss: 0.2253 - accuracy: 0.9321 - val_loss: 0.3947 - val_accuracy: 0.9565
              precision    recall  f1-score   support

         1.0       0.99      0.97      0.98      1351
         2.0       0.73      0.99      0.84        98
         3.0       1.00      0.96      0.98      1121
         4.0       0.73      0.94      0.82       128

    accuracy                           0.96      2698
   macro avg       0.86      0.96      0.91      2698
weighted avg       0.97      0.96      0.97      2698

Saved pruned Keras model to: /content/drive/MyDrive/DL/KD/base_cnn_student_strip.h5
INFO:tensorflow:Assets written to: /tmp/tmpdd0h164z/assets


Saved pruned TFLite model to: /content/drive/MyDrive/DL/KD/base_cnn_student_strip_tflite.h5
Size of gzipped baseline Keras model: 1457575.00 bytes
Size of gzipped pruned Keras model: 468626.00 bytes
Size of gzipped pruned TFlite model: 454350.00 bytes


In [ ]:
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=5,
)

# Distill teacher to student
distiller.fit(train_ds, epochs=7)

Epoch 1/7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


304/304 [==============================] - 322s 1s/step - sparse_categorical_accuracy: 0.7975 - student_loss: 0.6466 - distillation_loss: 0.0017
Epoch 2/7
304/304 [==============================] - 333s 1s/step - sparse_categorical_accuracy: 0.8680 - student_loss: 0.4348 - distillation_loss: 0.0011
Epoch 3/7
304/304 [==============================] - 328s 1s/step - sparse_categorical_accuracy: 0.8958 - student_loss: 0.3064 - distillation_loss: 7.7563e-04
Epoch 4/7
304/304 [==============================] - 328s 1s/step - sparse_categorical_accuracy: 0.9145 - student_loss: 0.2632 - distillation_loss: 6.5174e-04
Epoch 5/7
304/304 [==============================] - 326s 1s/step - sparse_categorical_accuracy: 0.9387 - student_loss: 0.1991 - distillation_loss: 4.9662e-04
Epoch 6/7
304/304 [==============================] - 320s 1s/step - sparse_categorical_accuracy: 0.9514 - student_loss: 0.1600 - distillation_loss: 3.9634e-04
Epoch 7/7
304/304 [==============================] - 320s 1s/ste

In [ ]:
##################################################################################

In [ ]:
# 128 student


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels,predictions))

              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99      1351
         2.0       0.77      0.99      0.87        98
         3.0       0.99      0.98      0.98      1121
         4.0       1.00      0.77      0.87       128

    accuracy                           0.98      2698
   macro avg       0.94      0.93      0.93      2698
weighted avg       0.98      0.98      0.98      2698



In [ ]:
#128 pruning


In [ ]:
student_model.save("/content/drive/MyDrive/DL/KD/128_5_7_cnn_student.h5")
pruned_keras_file = "/content/drive/MyDrive/DL/KD/128_5_7_cnn_student_strip.h5"
pruned_tflite_file = "/content/drive/MyDrive/DL/KD/128_5_7_cnn_student_strip_tflite.h5"
import tempfile


import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

end_step = 300*epochs
# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(student_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

predictions = np.array([])
labels =  np.array([])

for x, y in test_ds:
  predictions = np.concatenate([predictions, model_for_pruning.predict(x).argmax(axis =1)+1])
  labels = np.concatenate([labels, y.numpy()])


from sklearn.metrics import classification_report
print(classification_report(labels,predictions))


model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# _, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)


converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

# _, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


Epoch 1/2


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


304/304 [==============================] - 436s 1s/step - loss: 0.1193 - accuracy: 0.9634 - val_loss: 0.7413 - val_accuracy: 0.6970
Epoch 2/2
304/304 [==============================] - 405s 1s/step - loss: 0.1127 - accuracy: 0.9693 - val_loss: 0.3171 - val_accuracy: 0.8945
              precision    recall  f1-score   support

         1.0       1.00      0.89      0.94      1351
         2.0       0.30      1.00      0.46        98
         3.0       0.98      0.93      0.96      1121
         4.0       0.72      0.62      0.66       128

    accuracy                           0.90      2698
   macro avg       0.75      0.86      0.76      2698
weighted avg       0.95      0.90      0.92      2698

Saved pruned Keras model to: /content/drive/MyDrive/DL/KD/128_5_7_cnn_student_strip.h5
INFO:tensorflow:Assets written to: /tmp/tmptzanlkui/assets


Saved pruned TFLite model to: /content/drive/MyDrive/DL/KD/128_5_7_cnn_student_strip_tflite.h5
Size of gzipped baseline Keras model: 636053.00 bytes
Size of gzipped pruned Keras model: 230472.00 bytes
Size of gzipped pruned TFlite model: 210538.00 bytes


In [ ]:
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=5,
)

# Distill teacher to student
distiller.fit(train_ds, epochs=7)

Epoch 1/7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


304/304 [==============================] - 403s 1s/step - sparse_categorical_accuracy: 0.8386 - student_loss: 0.4901 - distillation_loss: 0.0013
Epoch 2/7
304/304 [==============================] - 345s 1s/step - sparse_categorical_accuracy: 0.9328 - student_loss: 0.2001 - distillation_loss: 5.1175e-04
Epoch 3/7
304/304 [==============================] - 343s 1s/step - sparse_categorical_accuracy: 0.9599 - student_loss: 0.1325 - distillation_loss: 3.3098e-04
Epoch 4/7
304/304 [==============================] - 345s 1s/step - sparse_categorical_accuracy: 0.9684 - student_loss: 0.1050 - distillation_loss: 2.5743e-04
Epoch 5/7
304/304 [==============================] - 343s 1s/step - sparse_categorical_accuracy: 0.9790 - student_loss: 0.0740 - distillation_loss: 1.7754e-04
Epoch 6/7
304/304 [==============================] - 355s 1s/step - sparse_categorical_accuracy: 0.9810 - student_loss: 0.0605 - distillation_loss: 1.5205e-04
Epoch 7/7
304/304 [==============================] - 368s 1s

In [ ]:
#128,256 student

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels,predictions))

              precision    recall  f1-score   support

         1.0       1.00      0.99      0.99      1351
         2.0       0.77      1.00      0.87        98
         3.0       1.00      0.96      0.98      1121
         4.0       0.82      1.00      0.90       128

    accuracy                           0.98      2698
   macro avg       0.90      0.99      0.94      2698
weighted avg       0.98      0.98      0.98      2698



In [ ]:
# 128,256 pruning


In [ ]:
student_model.save("/content/drive/MyDrive/DL/KD/128_256__5_7_cnn_student.h5")
pruned_keras_file = "/content/drive/MyDrive/DL/KD/128_256__5_7_cnn_student_strip.h5"
pruned_tflite_file = "/content/drive/MyDrive/DL/KD/128_256__5_7_cnn_student_strip_tflite.h5"
import tempfile


import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

end_step = 300*epochs
# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(student_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

predictions = np.array([])
labels =  np.array([])

for x, y in test_ds:
  predictions = np.concatenate([predictions, model_for_pruning.predict(x).argmax(axis =1)+1])
  labels = np.concatenate([labels, y.numpy()])


from sklearn.metrics import classification_report
print(classification_report(labels,predictions))


model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# _, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)


converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

# _, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


Epoch 1/2


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


304/304 [==============================] - 422s 1s/step - loss: 0.0288 - accuracy: 0.9915 - val_loss: 1.0486 - val_accuracy: 0.3080
Epoch 2/2
304/304 [==============================] - 409s 1s/step - loss: 0.0195 - accuracy: 0.9948 - val_loss: 0.0278 - val_accuracy: 0.9948
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00      1351
         2.0       0.96      1.00      0.98        98
         3.0       1.00      0.99      1.00      1121
         4.0       0.99      0.98      0.99       128

    accuracy                           1.00      2698
   macro avg       0.99      0.99      0.99      2698
weighted avg       1.00      1.00      1.00      2698

Saved pruned Keras model to: /content/drive/MyDrive/DL/KD/128_256__5_7_cnn_student_strip.h5
INFO:tensorflow:Assets written to: /tmp/tmphsgcr4h_/assets


Saved pruned TFLite model to: /content/drive/MyDrive/DL/KD/128_256__5_7_cnn_student_strip_tflite.h5
Size of gzipped baseline Keras model: 1633862.00 bytes
Size of gzipped pruned Keras model: 568672.00 bytes
Size of gzipped pruned TFlite model: 541135.00 bytes


In [ ]:
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=5,
)

# Distill teacher to student
distiller.fit(train_ds, epochs=7)

Epoch 1/7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


304/304 [==============================] - 421s 1s/step - sparse_categorical_accuracy: 0.9248 - student_loss: 0.2306 - distillation_loss: 5.9901e-04
Epoch 2/7
304/304 [==============================] - 388s 1s/step - sparse_categorical_accuracy: 0.9854 - student_loss: 0.0476 - distillation_loss: 1.1697e-04
Epoch 3/7
304/304 [==============================] - 386s 1s/step - sparse_categorical_accuracy: 0.9920 - student_loss: 0.0287 - distillation_loss: 6.8075e-05
Epoch 4/7
304/304 [==============================] - 383s 1s/step - sparse_categorical_accuracy: 0.9936 - student_loss: 0.0237 - distillation_loss: 5.7463e-05
Epoch 5/7
304/304 [==============================] - 385s 1s/step - sparse_categorical_accuracy: 0.9947 - student_loss: 0.0201 - distillation_loss: 4.4786e-05
Epoch 6/7
304/304 [==============================] - 387s 1s/step - sparse_categorical_accuracy: 0.9955 - student_loss: 0.0149 - distillation_loss: 3.5918e-05
Epoch 7/7
304/304 [==============================] - 386

In [1]:
#####################scratch##########################################

In [ ]:
#128 


In [ ]:
import numpy as np
predictions = np.array([])
labels =  np.array([])
for x, y in test_ds:
  temp = model.predict(x)
  predictions = np.concatenate([predictions, model.predict(x).argmax(axis =1)+1])
  labels = np.concatenate([labels, y.numpy()])


from sklearn.metrics import classification_report
print(classification_report(labels,predictions))

              precision    recall  f1-score   support

         1.0       0.89      1.00      0.94      1351
         2.0       0.92      0.70      0.80        98
         3.0       0.96      0.94      0.95      1121
         4.0       1.00      0.07      0.13       128

    accuracy                           0.92      2698
   macro avg       0.94      0.68      0.70      2698
weighted avg       0.92      0.92      0.90      2698



In [ ]:
##128,256

In [ ]:
import numpy as np
predictions = np.array([])
labels =  np.array([])
for x, y in test_ds:
  temp = model.predict(x)
  predictions = np.concatenate([predictions, model.predict(x).argmax(axis =1)+1])
  labels = np.concatenate([labels, y.numpy()])


from sklearn.metrics import classification_report
print(classification_report(labels,predictions))

              precision    recall  f1-score   support

         1.0       0.99      0.87      0.93      1351
         2.0       0.94      0.73      0.82        98
         3.0       0.79      0.99      0.88      1121
         4.0       0.96      0.17      0.29       128

    accuracy                           0.88      2698
   macro avg       0.92      0.69      0.73      2698
weighted avg       0.90      0.88      0.87      2698



In [ ]:
#prune


In [ ]:
predictions = np.array([])
labels =  np.array([])

for x, y in test_ds:
  predictions = np.concatenate([predictions, model_for_pruning.predict(x).argmax(axis =1)+1])
  labels = np.concatenate([labels, y.numpy()])


from sklearn.metrics import classification_report
print(classification_report(labels,predictions))

              precision    recall  f1-score   support

         1.0       0.85      1.00      0.92      1351
         2.0       0.97      0.76      0.85        98
         3.0       0.98      0.90      0.94      1121
         4.0       0.00      0.00      0.00       128

    accuracy                           0.90      2698
   macro avg       0.70      0.66      0.68      2698
weighted avg       0.87      0.90      0.88      2698



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
